In [12]:
import mediapipe as mp
import numpy as np
import cv2

In [13]:
mp_face_mesh = mp.solutions.face_mesh

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [14]:
src_img = cv2.imread("business-person.png")

In [25]:
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

In [15]:
def get_face_landmarks(image):
    face_mesh_algo= mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)



    # image = cv2.imread(img)
    image.flags.writeable = False
    image_src_gray = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh_algo.process(image_src_gray)

    face_landmark = results.multi_face_landmarks[0].landmark
    landmark_points = [(int(lm.x * image.shape[1]), int(lm.y * image.shape[0])) for lm in face_landmark]
    
    
    return landmark_points

In [79]:
def traingles_output(convexhull, landmarks_points, np_points):
    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)

    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        index_pt1 = extract_index_nparray(np.where((np_points == pt1).all(axis=1)))
        index_pt2 = extract_index_nparray(np.where((np_points == pt2).all(axis=1)))
        index_pt3 = extract_index_nparray(np.where((np_points == pt3).all(axis=1)))

        if index_pt1  and index_pt2  and index_pt3 :
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)

    return indexes_triangles

In [ ]:
def triangulation_of_images(triangle_index, landmark_points, img=None):
    tr1_pt1 = landmark_points[triangle_index[0]]
    tr1_pt2 = landmark_points[triangle_index[1]]
    tr1_pt3 = landmark_points[triangle_index[2]]
    triangle = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)

    rect = cv2.boundingRect(triangle)
    (x, y, w, h) = rect

    
    cropped_triangle = img[y: y + h, x: x + w]

    cropped_triangle_mask = np.zeros((h, w), np.uint8)

    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)

    cv2.fillConvexPoly(cropped_triangle_mask, points, 255)

    return points, cropped_triangle, cropped_triangle_mask, rect

In [ ]:
def warp_croped_triangle(rect, points1, points2, src_cropped_triangle, dest_cropped_triangle_mask):
    (x, y, w, h) = rect
    matrix = cv2.getAffineTransform(np.float32(points1), np.float32(points2))
    warped_triangle = cv2.warpAffine(src_cropped_triangle, matrix, (w, h))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=dest_cropped_triangle_mask)
    return warped_triangle

In [80]:
def add_piece_of_new_face(new_face, rect, warped_triangle):
    (x, y, w, h) = rect
    new_face_rect_area = new_face[y: y + h, x: x + w]
    new_face_rect_area_gray = cv2.cvtColor(new_face_rect_area, cv2.COLOR_BGR2GRAY)
    _, mask_triangles_designed = cv2.threshold(new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

    new_face_rect_area = cv2.add(new_face_rect_area, warped_triangle)
    new_face[y: y + h, x: x + w] = new_face_rect_area

In [81]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    
    _,frame= cap.read()

    src_img_gray = cv2.cvtColor(src_img,cv2.COLOR_BGR2GRAY)

    src_img_mask= np.zeros_like(src_img_gray)
    src_img_landmark_points = get_face_landmarks(src_img)
        
    src_img_np_landmarks = np.array(src_img_landmark_points)
    src_img_convexhull = cv2.convexHull(src_img_np_landmarks)
    src_img_copy = src_img.copy()

    # cv2.polylines(src_img_copy,[src_img_convexhull],True,(255,0,0),3)
    cv2.fillConvexPoly(src_img_mask, src_img_convexhull,255)

    src_img_1 = cv2.bitwise_and(src_img, src_img, mask=src_img_mask)



    #  Delauney Triangulation
    rect = cv2.boundingRect(src_img_convexhull)  # it is becauase we want to do triangulation on specific area .
    (xs,ys,ws,hs) = rect
    # cv2.rectangle(src_img_copy,(xs,ys),(xs+ws,ys+hs),(0,255,0))
    subdiv= cv2.Subdiv2D(rect)   # gives triangles cordinates dividing the convewx hull based on landmarks points.
    subdiv.insert(src_img_landmark_points)
    src_img_triangles = subdiv.getTriangleList()
    src_img_triangles = np.array(src_img_triangles, dtype=np.int32)

    src_img_triangle_indices = []
    for t in src_img_triangles:
        pt1 = (t[0],t[1])
        pt2 = (t[2],t[3])
        pt3 = (t[4],t[5])
        
        index_pt1 = np.where((src_img_np_landmarks == pt1).all(axis=1))
        index_pt1= extract_index_nparray(index_pt1)
    
        index_pt2 = np.where((src_img_np_landmarks == pt2).all(axis=1))
        index_pt2= extract_index_nparray(index_pt2)
    
        index_pt3 = np.where((src_img_np_landmarks == pt3).all(axis=1))
        index_pt3= extract_index_nparray(index_pt3)
        
        if index_pt1 and index_pt2 and index_pt3:
            triangle = [index_pt1, index_pt2, index_pt3]
            src_img_triangle_indices.append(triangle)

        # cv2.line(src_img_copy, pt1, pt2,(235,12,35),2)
        # cv2.line(src_img_copy, pt2, pt3,(235,12,35),2)
        # cv2.line(src_img_copy, pt1, pt3,(235,12,35),2)


    #===============================================================================
    #  FOR Second Image
    final_img = frame
    final_img = cv2.resize(final_img, (1200, 1000))
    final_img_copy= final_img.copy()
    final_image_new_face = np.zeros_like(final_img)


    final_img_gray = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
    final_img_mask = np.zeros_like(final_img_gray)
        
    final_img_landmark_points = get_face_landmarks(final_img)
        
    final_img_np_landmarks = np.array(final_img_landmark_points)
    final_img_convexhull = cv2.convexHull(final_img_np_landmarks)
        
        
    # cv2.polylines(final_img_copy,[final_img_convexhull],True,(225,0,0),3)
    cv2.fillConvexPoly(final_img_mask, final_img_convexhull,255)

    final_img_1 = cv2.bitwise_and(final_img, final_img, mask=final_img_mask)

    rect_final_img = cv2.boundingRect(final_img_convexhull)
    (xf,yf,wf,hf) = rect_final_img
    # cv2.rectangle(final_img_copy,(xf,yf),(xf+wf,yf+hf),(255,0,0),2)
    final_img_subdiv= cv2.Subdiv2D(rect_final_img)
    final_img_subdiv.insert(final_img_landmark_points)
    final_img_triangles = final_img_subdiv.getTriangleList()
    final_img_triangles = np.array(final_img_triangles, dtype=np.int32)
    
    
    final_img_triangle_indices = []
    for t in final_img_triangles:
        pt1 = (t[0],t[1])
        pt2 = (t[2],t[3])
        pt3 = (t[4],t[5])
        
        index_pt1 = np.where((src_img_np_landmarks == pt1).all(axis=1))
        index_pt1= extract_index_nparray(index_pt1)
    
        index_pt2 = np.where((src_img_np_landmarks == pt2).all(axis=1))
        index_pt2= extract_index_nparray(index_pt2)
    
        index_pt3 = np.where((src_img_np_landmarks == pt3).all(axis=1))
        index_pt3= extract_index_nparray(index_pt3)
        
        if index_pt1 and index_pt2 and index_pt3:
            triangle = [index_pt1, index_pt2, index_pt3]
            final_img_triangle_indices.append(triangle)

        # cv2.line(final_img_copy, pt1, pt2,(235,12,35),2)
        # cv2.line(final_img_copy, pt2, pt3,(235,12,35),2)
        # cv2.line(final_img_copy, pt1, pt3,(235,12,35),2)    




    for triangle_index in src_img_triangle_indices:
        
        # Triangulation of the src_img    
        spt1= src_img_landmark_points[triangle_index[0]]
        spt2 = src_img_landmark_points[triangle_index[1]]
        spt3= src_img_landmark_points[triangle_index[2]]
        
        src_triangle1 = np.array([spt1, spt2, spt3],np.int32)
        rect1 = cv2.boundingRect(src_triangle1)
        (xrect1, yrect1, wrect1,hrect1) = rect1
        src_cropperd_triangle =  src_img[yrect1:yrect1+hrect1,xrect1:xrect1+wrect1]
        src_cropperd_triangle_mask= np.zeros((hrect1,wrect1),np.uint8)
        points1 = np.array([[spt1[0]-xrect1,spt1[1]-yrect1],
                            [spt2[0]-xrect1,spt2[1]-yrect1],
                            [spt3[0]-xrect1,spt3[1]-yrect1]
                            ],np.int32)
        
        cv2.fillConvexPoly(src_cropperd_triangle_mask,points1,255)
        src_cropperd_triangle = cv2.bitwise_and(src_cropperd_triangle,src_cropperd_triangle,mask=src_cropperd_triangle_mask)
        
        
        # cv2.rectangle(src_img_copy,(xrect1,yrect1),(xrect1+wrect1,yrect1+hrect1),(0,255,0),2)
        
        
        # cv2.line(src_img_copy, spt1, spt2,(235,12,35),2)
        # cv2.line(src_img_copy, spt2, spt3,(235,12,35),2)
        # cv2.line(src_img_copy, spt1, spt3,(235,12,35),2)
        
        
        #Triangulation of final_img based on src_img 
        fpt1= final_img_landmark_points[triangle_index[0]]
        fpt2 = final_img_landmark_points[triangle_index[1]]
        fpt3= final_img_landmark_points[triangle_index[2]]
        
        
        final_triangle1 = np.array([fpt1, fpt2, fpt3],np.int32)
        rect2 = cv2.boundingRect(final_triangle1)
        (xrect2, yrect2, wrect2,hrect2) = rect2
        final_cropperd_triangle =  final_img[yrect2:yrect2+hrect2,xrect2:xrect2+wrect2]
        final_cropperd_triangle_mask= np.zeros((hrect2,wrect2),np.uint8)
        points2 = np.array([[fpt1[0]-xrect2,fpt1[1]-yrect2],
                            [fpt2[0]-xrect2,fpt2[1]-yrect2],
                            [fpt3[0]-xrect2,fpt3[1]-yrect2]
                            ],np.int32)
        
        cv2.fillConvexPoly(final_cropperd_triangle_mask,points2,255)
        final_cropperd_triangle = cv2.bitwise_and(final_cropperd_triangle,final_cropperd_triangle,mask=final_cropperd_triangle_mask)
        
        # cv2.rectangle(final_img_copy,(xrect2,yrect2),(xrect2+wrect2,yrect2+hrect2),(0,255,0),2)
        
        
        
        # cv2.line(final_img_copy, fpt1, fpt2,(235,12,35),2)
        # cv2.line(final_img_copy, fpt2, fpt3,(235,12,35),2)
        # cv2.line(final_img_copy, fpt1, fpt3,(235,12,35),2) 
        
        # Warap Triangles 
        
        points1 = np.float32(points1)
        points2 = np.float32(points2)
        
        M = cv2.getAffineTransform(points1,points2)
        
        # print(M)
        
        wraped_triangle =  cv2.warpAffine(src_cropperd_triangle,M,(wrect2,hrect2))
        
        
        
        # Final REconstruction
        height, width, channels = final_img_copy.shape
        new_face = np.zeros((height, width, channels), np.uint8)
        (x, y, w, h) = rect2
        new_face_rect_area = new_face[y: y + h, x: x + w]
        new_face_rect_area_gray = cv2.cvtColor(new_face_rect_area, cv2.COLOR_BGR2GRAY)
        _, mask_triangles_designed = cv2.threshold(new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
        warped_triangle = cv2.bitwise_and(wraped_triangle, wraped_triangle, mask=mask_triangles_designed)

        new_face_rect_area = cv2.add(new_face_rect_area, warped_triangle)
        new_face[y: y + h, x: x + w] = new_face_rect_area
        
        
        
        face_mask = final_img_gray
        head_mask = cv2.fillConvexPoly(face_mask, final_img_convexhull, 255)
        face_mask = cv2.bitwise_not(head_mask)

        head_without_face = cv2.bitwise_and(final_img_copy, final_img_copy, mask=face_mask)
        result = cv2.add(head_without_face, new_face)

        (x, y, w, h) = cv2.boundingRect(final_img_convexhull)
        center_face = (int((x + x + w) / 2), int((y + y + h) / 2))

        output= cv2.seamlessClone(result, final_img_copy, head_mask, center_face, cv2.MIXED_CLONE)
        # break
    

    #Swapping the final final_cropperd_triangle

    result = cv2.medianBlur(output, 3)
    h, w, _ = frame.shape
    rate = width / w

    cv2.imshow("Source image", cv2.resize(frame, (int(w * rate), int(h * rate))))
    cv2.imshow("New face", new_face)
    # result1.write(result)
    cv2.imshow("Result", result)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cv2.destroyAllWindows()







    # # cv2.imshow("final_imgconvexhull",final_img_copy)
    # # cv2.imshow("final_image_extracted", final_img_1)

    # cv2.imshow("Src_img", src_img_copy)
    # cv2.imshow("final_img",final_img_copy)
    # # cv2.imshow("finalFace", final_image_new_face)
    # cv2.imshow("result", output)
    # # cv2.imshow("Face extracted ", src_img_1)
    # # cv2.imshow("croped_trngl_src",src_cropperd_triangle)
    # # cv2.imshow("croped_trngl_final",final_cropperd_triangle)
    # # cv2.imshow("wrapped triangle",wraped_triangle)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


C:\Users\bhard\AppData\Roaming\Python\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\matrix.cpp:809: error: (-215:Assertion failed) 0 <= roi.x && 0 <= roi.width && roi.x + roi.width <= m.cols && 0 <= roi.y && 0 <= roi.height && roi.y + roi.height <= m.rows in function 'cv::Mat::Mat'


In [78]:


# Read the source image and convert to grayscale
src_img = cv2.imread("business-person.png")
src_img_gray = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)

# Create a mask for the source image
src_img_mask = np.zeros_like(src_img_gray)
src_img_landmark_points = get_face_landmarks(src_img)
src_img_np_landmarks = np.array(src_img_landmark_points)
src_img_convexhull = cv2.convexHull(src_img_np_landmarks)
cv2.fillConvexPoly(src_img_mask, src_img_convexhull, 255)
src_img_1 = cv2.bitwise_and(src_img, src_img, mask=src_img_mask)

# Delaunay Triangulation for the source image
rect = cv2.boundingRect(src_img_convexhull)
subdiv = cv2.Subdiv2D(rect)
subdiv.insert(src_img_landmark_points)
src_img_triangles = subdiv.getTriangleList()
src_img_triangles = np.array(src_img_triangles, dtype=np.int32)
src_img_triangle_indices = []
for t in src_img_triangles:
    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])
    
    index_pt1 = np.where((src_img_np_landmarks == pt1).all(axis=1))
    index_pt1 = extract_index_nparray(index_pt1)
    index_pt2 = np.where((src_img_np_landmarks == pt2).all(axis=1))
    index_pt2 = extract_index_nparray(index_pt2)
    index_pt3 = np.where((src_img_np_landmarks == pt3).all(axis=1))
    index_pt3 = extract_index_nparray(index_pt3)
    
    if index_pt1 and index_pt2 and index_pt3:
        triangle = [index_pt1, index_pt2, index_pt3]
        src_img_triangle_indices.append(triangle)

# Read the final image and convert to grayscale
final_img = cv2.imread("elon_musk_royal_society.jpg")
final_img = cv2.resize(final_img, (1200, 1000))
final_img_copy = final_img.copy()
final_image_new_face = np.zeros_like(final_img)
final_img_gray = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
final_img_mask = np.zeros_like(final_img_gray)
final_img_landmark_points = get_face_landmarks(final_img)
final_img_np_landmarks = np.array(final_img_landmark_points)
final_img_convexhull = cv2.convexHull(final_img_np_landmarks)
cv2.fillConvexPoly(final_img_mask, final_img_convexhull, 255)
final_img_1 = cv2.bitwise_and(final_img, final_img, mask=final_img_mask)

# Delaunay Triangulation for the final image
rect_final_img = cv2.boundingRect(final_img_convexhull)
final_img_subdiv = cv2.Subdiv2D(rect_final_img)
final_img_subdiv.insert(final_img_landmark_points)
final_img_triangles = final_img_subdiv.getTriangleList()
final_img_triangles = np.array(final_img_triangles, dtype=np.int32)
final_img_triangle_indices = []
for t in final_img_triangles:
    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])
    
    index_pt1 = np.where((final_img_np_landmarks == pt1).all(axis=1))
    index_pt1 = extract_index_nparray(index_pt1)
    index_pt2 = np.where((final_img_np_landmarks == pt2).all(axis=1))
    index_pt2 = extract_index_nparray(index_pt2)
    index_pt3 = np.where((final_img_np_landmarks == pt3).all(axis=1))
    index_pt3 = extract_index_nparray(index_pt3)
    
    if index_pt1 and index_pt2 and index_pt3:
        triangle = [index_pt1, index_pt2, index_pt3]
        final_img_triangle_indices.append(triangle)

# Warp triangles and blend seamlessly
for triangle_index in src_img_triangle_indices:
    spt1 = src_img_landmark_points[triangle_index[0]]
    spt2 = src_img_landmark_points[triangle_index[1]]
    spt3 = src_img_landmark_points[triangle_index[2]]
    
    src_triangle1 = np.array([spt1, spt2, spt3], np.int32)
    rect1 = cv2.boundingRect(src_triangle1)
    (xrect1, yrect1, wrect1, hrect1) = rect1
    src_cropperd_triangle = src_img[yrect1:yrect1+hrect1, xrect1:xrect1+wrect1]
    src_cropperd_triangle_mask = np.zeros((hrect1, wrect1), np.uint8)
    points1 = np.array([[spt1[0]-xrect1, spt1[1]-yrect1],
                        [spt2[0]-xrect1, spt2[1]-yrect1],
                        [spt3[0]-xrect1, spt3[1]-yrect1]], np.int32)
    cv2.fillConvexPoly(src_cropperd_triangle_mask, points1, 255)
    src_cropperd_triangle = cv2.bitwise_and(src_cropperd_triangle, src_cropperd_triangle, mask=src_cropperd_triangle_mask)
    
    fpt1 = final_img_landmark_points[triangle_index[0]]
    fpt2 = final_img_landmark_points[triangle_index[1]]
    fpt3 = final_img_landmark_points[triangle_index[2]]
    
    final_triangle1 = np.array([fpt1, fpt2, fpt3], np.int32)
    rect2 = cv2.boundingRect(final_triangle1)
    (xrect2, yrect2, wrect2, hrect2) = rect2
    final_cropperd_triangle = final_img[yrect2:yrect2+hrect2, xrect2:xrect2+wrect2]
    final_cropperd_triangle_mask = np.zeros((hrect2, wrect2), np.uint8)
    points2 = np.array([[fpt1[0]-xrect2, fpt1[1]-yrect2],
                        [fpt2[0]-xrect2, fpt2[1]-yrect2],
                        [fpt3[0]-xrect2, fpt3[1]-yrect2]], np.int32)
    cv2.fillConvexPoly(final_cropperd_triangle_mask, points2, 255)
    final_cropperd_triangle = cv2.bitwise_and(final_cropperd_triangle, final_cropperd_triangle, mask=final_cropperd_triangle_mask)
    
    points1 = np.float32(points1)
    points2 = np.float32(points2)
    M = cv2.getAffineTransform(points1, points2)
    wraped_triangle = cv2.warpAffine(src_cropperd_triangle, M, (wrect2, hrect2))
    
    # Dilate the warped triangle to remove gaps
    dilate_kernel = np.ones((3, 3), np.uint8)
    wraped_triangle = cv2.dilate(wraped_triangle, dilate_kernel, iterations=1)
    
    triangle_area = final_image_new_face[yrect2:yrect2+hrect2, xrect2:xrect2+wrect2]
    triangle_area = cv2.add(triangle_area, wraped_triangle)
    final_image_new_face[yrect2:yrect2+hrect2, xrect2:xrect2+wrect2] = triangle_area

# Apply seamless cloning using Poisson blending
final_image_new_face_gray = cv2.cvtColor(final_image_new_face, cv2.COLOR_BGR2GRAY)
_, background = cv2.threshold(final_image_new_face_gray, 1, 255, cv2.THRESH_BINARY_INV)
background = cv2.bitwise_and(final_img_copy, final_img_copy, mask=background)
result = cv2.add(background, final_image_new_face)

# Blur the mask to smooth the edges
blurred_mask = cv2.GaussianBlur(final_img_mask, (15, 15), 10)
center_point = (rect_final_img[0] + rect_final_img[2]//2, rect_final_img[1] + rect_final_img[3]//2)
output = cv2.seamlessClone(final_image_new_face, final_img, blurred_mask, center_point, cv2.MIXED_CLONE)

cv2.imshow("Result", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [11]:
cap= cv2.VideoCapture(0)

while cap.isOpened():
    _,final_img = cap.read()
    final_img_copy= final_img.copy()

    final_img_gray = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
    final_img_mask = np.zeros_like(final_img_gray)
    
    final_img_landmark_points = get_face_landmarks(final_img)
    
    final_img_np_landmarks = np.array(final_img_landmark_points)
    final_img_convexhull = cv2.convexHull(final_img_np_landmarks)
    
    
    cv2.polylines(final_img_copy,[final_img_convexhull],True,(225,0,0),3)
    
    cv2.imshow("ConvexHull",final_img_copy)
    
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    
    